In [29]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [27]:
%sendtofile config.txt

connection3  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection2  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection0  DoESLiverpool,decafbad00,mosquitto.doesliverpool.xyz
connection1  BT-STCPTN,FNPmKv4V7hMyJy,mosquitto.doesliverpool.xyz
pinled       2
pinscl       13
pinsda       14
uartbus      2
wdtsecs      300
position     dusty

Sent 10 lines (358 bytes) to config.txt.


In [13]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 647 lines (23171 bytes) to mqtt_as.py.


In [30]:
%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from stdmqttas import shortmac, topicstem, itertools_count
from stdmqttas import pinled, pinledOnvalue, flashpinled

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)
print("topicstem", topicstem)

topicplantower = topicstem+"/plantower"
topicscd30 = topicstem+"/scd30"
topicmhz = topicstem+"/MHZ19"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
sensorposition = fconfig.get("position", "")

uart1 = machine.UART(1, baudrate=9600, tx=33, rx=32)
uart2 = machine.UART(2, baudrate=9600)
i2c = machine.SoftI2C(scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))

async def WatchDogtask(client, wdtsecs):
    await asyncio.sleep_ms(20*1000)
    print("Starting Watchdog timer for", wdtsecs, "seconds")
    wd = machine.WDT(timeout=int(wdtsecs*1000))
    while True:
        await asyncio.sleep_ms(1*1000)
        if client.isconnected():
            wd.feed()
            pinled.value(pinledOnvalue)
            await asyncio.sleep_ms(50)
            pinled.value(not pinledOnvalue)
            await asyncio.sleep_ms(9*1000)
        elif pinled:
            pinled.value(pinledOnvalue)
            await asyncio.sleep_ms(1*1000)
            pinled.value(not pinledOnvalue)
            print("Not feeding watchdog")

            
async def plantowertask(client, uart):
    try:
        for n in itertools_count():
            i = 0
            checksum = 0
            while True:
                await asyncio.sleep_ms(10)
                u1 = uart.read(1)
                if u1 == None:
                    await asyncio.sleep_ms(100)
                    continue
                checksum += u1[0]
                if u1[0] == b"BM"[i]:
                    i += 1
                    if i == 2:
                        break
                else:
                    i = 0
                    checksum = 0
            sleng = uart.read(2)
            leng = struct.unpack(">H", sleng)[0]
            checksum += sum(sleng)
            fdata = uart.read(leng)
            data = struct.unpack(">HHHHHHHHHHHHHH", fdata)
            checksum += sum(fdata[:-2])
            if checksum == data[13]:
                print(n, data[:13])
                if client.isconnected():
                    pinled.value(1)
                    await client.publish(topicplantower, "%d %d %d %d %d %d %d %d %d %d %d %d %d"%data[:13])
                    pinled.value(0)
    except OSError as e:
        print("plantowertask", e)
        flashpinled(5, 300, 100)
        aloop.create_task(plantowertask(client))

async def MHZ19_task(client, uart):
    try:
        uart.write(bytes([0xFF, 0x01, 0x79, 0x00, 0x00, 0x00, 0x00, 0x00, 0x86]))
        while True:
            await asyncio.sleep_ms(2000)
            uart.write(bytes([0xFF, 0x01, 0x86, 0x00, 0x00, 0x00, 0x00, 0x00, 0x79]))
            await asyncio.sleep_ms(100)
            s = uart.read()
            if s and len(s) == 9 and s[0] == 0xFF and s[1] == 0x86:
                v = s[2]*256 + s[3]
                print(topicmhz, v)
                if client.isconnected():
                    await client.publish(topicmhz, "%d %s"%(v, sensorposition))
                    pinled.value(pinledOnvalue)
                    await asyncio.sleep_ms(50)
                    pinled.value(not pinledOnvalue)
    except OSError as e:
        print("MHZ19_task", e)
    await asyncio.sleep_ms(5000)
    aloop.create_task(MHZ19_task(client, uart))
        

def crc8(b):
    crc = 0xFF
    for c in b:
        crc ^= c
        for i in range(8):
            crc = 0xFF & (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1))
    return crc
        
async def SCD30task(client, i2c, topicscd30):
    try:
        i2c.writeto(0x61, bytes([0x02, 0x04]))   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, bytes([0x02, 0x02]))  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == bytes([0x00, 0x01]):
                y = i2c.writeto(0x61, bytes([0x03, 0x00]))  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                crcvalid = (len(x) == 18) and (crc8(x[0:2]) == x[2]) and (crc8(x[3:5]) == x[5]) and (crc8(x[6:8]) == x[8]) and (crc8(x[9:11]) == x[11]) and (crc8(x[12:14]) == x[14]) and (crc8(x[15:17]) == x[17])
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid, crcvalid)
                if client.isconnected() and crcvalid:
                    await client.publish(topicscd30, "%f %f %f %s"%(co2, temp, humid, sensorposition))
                    pinled.value(pinledOnvalue)
                    await asyncio.sleep_ms(50)
                    pinled.value(not pinledOnvalue)
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", topicscd30, e)
    await asyncio.sleep_ms(5000)
    aloop.create_task(SCD30task(client, i2c, topicscd30))
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(SCD30task(client, i2c, topicscd30))
aloop.create_task(plantowertask(client, uart2))
aloop.create_task(MHZ19_task(client, uart1))
aloop.create_task(mqttconnecttask(client))
if "wdtsecs" in fconfig:
    aloop.create_task(WatchDogtask(client, float(fconfig["wdtsecs"])))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 171 lines (6798 bytes) to main.py.
